# Analyses

## library comparison analyses 

## data setup 

In [ ]:
%%bash

## Convert text files exported from MEGAN into QZA files (QIIME2 input files) 
## For the feature-level tables 

biom convert -i Feature_Analysis/absolute_all_merged_features_16S.txt \
-o Feature_Analysis/absolute_all_merged_features_16S-hdf5.biom \
--table-type="OTU table" --to-hdf5

qiime tools import \
  --input-path Feature_Analysis/absolute_all_merged_features_16S-hdf5.biom \
  --type 'FeatureTable[Frequency]' \
  --input-format BIOMV210Format \
  --output-path Feature_Analysis/absolute_all_merged_features.qza

qiime feature-table summarize \
  --i-table Feature_Analysis/absolute_all_merged_features.qza \
  --o-visualization Feature_Analysis/absolute_all_merged_features.qzv
#A13208_frSF2_55c_SortMeRNA-100k-0-1:	14783

## For the genus-level tables 

biom convert -i Genus_Analysis/absolute_all_merged_genera_16S.txt \
-o Genus_Analysis/absolute_all_merged_genera_16S-hdf5.biom \
--table-type="OTU table" --to-hdf5

qiime tools import \
  --input-path Genus_Analysis/absolute_all_merged_genera_16S-hdf5.biom \
  --type 'FeatureTable[Frequency]' \
  --input-format BIOMV210Format \
  --output-path Genus_Analysis/absolute_all_merged_genera_16S.qza

qiime feature-table summarize \
  --i-table Genus_Analysis/absolute_all_merged_genera_16S.qza \
  --o-visualization Genus_Analysis/absolute_all_merged_genera_16S.qzv

## alpha diversity analysis 

In [ ]:
%%bash

## perform alpha diversity analysis 
## these are the results that were used for the Kruskal-Wallis statistics and boxplots 
qiime diversity alpha-rarefaction \
--i-table Feature_Analysis/absolute_all_merged_FEATURES-sample_filtered.qza \
--p-max-depth 9980 \
--p-metrics observed_features \
--m-metadata-file 16S-enrichment-MappingFile_15JUN23.txt \
--o-visualization Feature_Analysis/rarefaction-16S-Enrichment-Shotgun-FILTERED-features.qzv

qiime diversity alpha-rarefaction \
--i-table Genus_Analysis/absolute_all_merged_GENERA_16S_filtered.qza \
--p-max-depth 9731 \
--p-metrics observed_features \
--m-metadata-file 16S-enrichment-MappingFile_15JUN23.txt \
--o-visualization Genus_Analysis/rarefaction-16S-Enrichment-Shotgun-FILTERED-genus.qzv

## export rarefied tables to txt files for analysis in R 
qiime tools export   \
--input-path Feature_Analysis/absolute_all_merged_FEATURES-sample_filtered.qza \
--output-path Feature_Analysis/Exported-Feature_Analysis

biom convert -i Feature_Analysis/Exported-Feature_Analysis/feature-table.biom \
-o Feature_Analysis/Exported-Feature_Analysis/absolute_all_merged_FEATURES.txt \
--to-tsv

qiime tools export   \
--input-path Genus_Analysis/absolute_all_merged_GENERA_16S_filtered.qza \
--output-path Genus_Analysis/Exported-Genus_Analysis

biom convert -i Genus_Analysis/Exported-Genus_Analysis/feature-table.biom \
-o Genus_Analysis/Exported-Genus_Analysis/absolute_all_merged_GENERA.txt \
--to-tsv

## beta diversity analysis 

In [ ]:
%%bash

## set up variables 
NAME=All-FEATURES
QZA=absolute_all_merged_FEATURES-sample_filtered.qza
METADATA=16S-enrichment-MappingFile_15JUN23.txt

## Feature analysis 
## perform Aitchison analysis first 
qiime deicode rpca \
    --i-table Feature_Analysis/$QZA \
    --p-min-feature-count 10 \
    --o-biplot Feature_Analysis/${NAME}-ordination.qza \
    --o-distance-matrix Feature_Analysis/${NAME}-aitchison-distance.qza

declare -a StringArray=("LibraryType")
for category in ${StringArray[@]}; do
qiime diversity beta-group-significance \
  --i-distance-matrix Feature_Analysis/${NAME}-aitchison-distance.qza \
  --m-metadata-file $METADATA \
  --m-metadata-column "$category" \
  --o-visualization Feature_Analysis/${NAME}-$category-significance.qzv \
  --p-pairwise
done

qiime emperor biplot \
    --i-biplot Feature_Analysis/${NAME}-ordination.qza \
    --m-sample-metadata-file $METADATA \
    --o-visualization Feature_Analysis/${NAME}-biplot-4-features.qzv \
    --p-number-of-features 4

## perform Bray-Curtis analysis 
qiime diversity core-metrics \
--i-table Feature_Analysis/$QZA \
--p-sampling-depth 9980 \
--m-metadata-file $METADATA \
--output-dir Feature_Analysis/core-metrics-${NAME}
## rarefied tables to 9980 because sample A12017_EuroHG2-RefSeqGCS had the lowest counts 

qiime diversity beta-group-significance \
  --i-distance-matrix Feature_Analysis/core-metrics-${NAME}/bray_curtis_distance_matrix.qza \
  --m-metadata-file $METADATA \
  --m-metadata-column LibraryType \
  --o-visualization Feature_Analysis/core-metrics-${NAME}/${NAME}-LibraryType-features-bray-curtis.qzv \
  --p-pairwise

## Genus analysis 

NAME=Genus
QZA=absolute_all_merged_GENERA_16S_filtered.qza

#Aitchison analysis 
qiime deicode rpca \
    --i-table Genus_Analysis/$QZA \
    --p-min-feature-count 10 \
    --o-biplot Genus_Analysis/${NAME}-ordination.qza \
    --o-distance-matrix Genus_Analysis/${NAME}-aitchison-distance.qza

declare -a StringArray=("LibraryType")
for category in ${StringArray[@]}; do

qiime diversity beta-group-significance \
  --i-distance-matrix Genus_Analysis/${NAME}-aitchison-distance.qza \
  --m-metadata-file $METADATA \
  --m-metadata-column "$category" \
  --o-visualization Genus_Analysis/${NAME}-$category-significance.qzv \
  --p-pairwise
done

qiime emperor biplot \
    --i-biplot Genus_Analysis/${NAME}-ordination.qza \
    --m-sample-metadata-file $METADATA \
    --o-visualization Genus_Analysis/${NAME}-biplot-4-features.qzv \
    --p-number-of-features 4

## Bray-Curtis analysis 
qiime diversity core-metrics \
--i-table Genus_Analysis/$QZA \
--p-sampling-depth 9731 \
--m-metadata-file $METADATA \
--output-dir Genus_Analysis/core-metrics-${NAME}
## A13208_AfrSF2-100k-SortMeRNA-Naive - 9731

qiime diversity beta-group-significance \
  --i-distance-matrix Genus_Analysis/core-metrics-${NAME}/bray_curtis_distance_matrix.qza \
  --m-metadata-file $METADATA \
  --m-metadata-column LibraryType \
  --o-visualization Genus_Analysis/core-metrics-${NAME}/${NAME}-LibraryType-features-bray-curtis.qzv \
  --p-pairwise

## temperature comparison analyses 

### data setup  

In [ ]:
%%bash

QZA=AbsoluteAll-16S-vs-RefSeqGCS-all-features.qza
NAME=Shotgun-all-features
METADATA=MappingFile.txt

qiime feature-table filter-samples \
--i-table 16S-enrichment-amplicon-shotgun-genus.qza \
--m-metadata-file 16S-enrichment-MappingFile_12Dec22.txt \
--p-where "LibraryType IN ('Enriched', '16S')" \
--o-filtered-table Samples-genus-Enriched-16S.qza

QZA=16S-AbsoluteAll-16S-vs-RefSeqGCS-Genera.qza
NAME=Shotgun-Genera

qiime feature-table filter-samples \
--i-table 16S-enrichment-amplicon-shotgun-features.qza \
--m-metadata-file 16S-enrichment-MappingFile_12Dec22.txt \
--p-where "LibraryType IN ('Enriched', '16S')" \
--o-filtered-table Samples-features-Enriched-16S.qza

### alpha diversity analysis 

In [ ]:
%%bash

qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-16S-enrichment-amplicon-shotgun-genus/observed_features_vector.qza \
  --m-metadata-file MappingFile.txt \
  --o-visualization core-metrics-16S-enrichment-amplicon-shotgun-genus/alpha-diversity-observed-genera-significance.qzv

qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-16S-enrichment-amplicon-shotgun-features/observed_features_vector.qza \
  --m-metadata-file MappingFile.txt \
  --o-visualization core-metrics-16S-enrichment-amplicon-shotgun-features/alpha-diversity-observed-features-significance.qzv


### beta diversity analysis 

In [ ]:
%%bash

##Features 

NAME=All-features
QZA=AbsoluteAll-16S-vs-RefSeqGCS-all-features.qza
METADATA=MappingFile.txt

mkdir Taxonomic-${NAME}

#Aitchison analysis 
qiime deicode rpca \
    --i-table Taxonomic-${NAME}/${NAME}_$QZA \
    --p-min-feature-count 10 \
    --o-biplot Taxonomic-${NAME}/${NAME}-ordination.qza \
    --o-distance-matrix Taxonomic-${NAME}/${NAME}-aitchison-distance.qza

declare -a StringArray=("Temperature")
for category in ${StringArray[@]}; do
qiime diversity beta-group-significance \
  --i-distance-matrix Taxonomic-${NAME}/${NAME}-aitchison-distance.qza \
  --m-metadata-file $METADATA \
  --m-metadata-column "$category" \
  --o-visualization Taxonomic-${NAME}/${NAME}-$category-significance.qzv \
  --p-pairwise
done

qiime emperor biplot \
    --i-biplot Taxonomic-${NAME}/${NAME}-ordination.qza \
    --m-sample-metadata-file $METADATA \
    --o-visualization Taxonomic-${NAME}/${NAME}-biplot-4-features.qzv \
    --p-number-of-features 4

#Bray-Curtis analysis 
qiime diversity core-metrics \
--i-table Taxonomic-${NAME}/${NAME}_$QZA \
--p-sampling-depth 14783 \
--m-metadata-file $METADATA \
--output-dir Taxonomic-${NAME}/core-metrics-${NAME}

qiime diversity beta-group-significance \
  --i-distance-matrix Taxonomic-${NAME}/core-metrics-${NAME}/bray_curtis_distance_matrix.qza \
  --m-metadata-file $METADATA \
  --m-metadata-column Temperature \
  --o-visualization Taxonomic-${NAME}/core-metrics-${NAME}/${NAME}-Temperature-features-bray-curtis.qzv \
  --p-pairwise

## Genus analysis 

NAME=16S-genus
QZA=AbsoluteAll-16S-vs-RefSeqGCS-all-features.qza

#Aitchison analysis 
qiime deicode rpca \
    --i-table Taxonomic-${NAME}/${NAME}_$QZA \
    --p-min-feature-count 10 \
    --o-biplot Taxonomic-${NAME}/${NAME}-ordination.qza \
    --o-distance-matrix Taxonomic-${NAME}/${NAME}-aitchison-distance.qza

declare -a StringArray=("Temperature")
for category in ${StringArray[@]}; do
qiime diversity beta-group-significance \
  --i-distance-matrix Taxonomic-${NAME}/${NAME}-aitchison-distance.qza \
  --m-metadata-file $METADATA \
  --m-metadata-column "$category" \
  --o-visualization Taxonomic-${NAME}/${NAME}-$category-significance.qzv \
  --p-pairwise
done

qiime emperor biplot \
    --i-biplot Taxonomic-${NAME}/${NAME}-ordination.qza \
    --m-sample-metadata-file $METADATA \
    --o-visualization Taxonomic-${NAME}/${NAME}-biplot-4-features.qzv \
    --p-number-of-features 4

#Bray-Curtis analysis 
qiime diversity core-metrics \
--i-table Taxonomic-${NAME}/${NAME}_$QZA \
--p-sampling-depth 14783 \
--m-metadata-file $METADATA \
--output-dir Taxonomic-${NAME}/core-metrics-${NAME}

qiime diversity beta-group-significance \
  --i-distance-matrix Taxonomic-${NAME}/core-metrics-${NAME}/bray_curtis_distance_matrix.qza \
  --m-metadata-file $METADATA \
  --m-metadata-column Temperature \
  --o-visualization Taxonomic-${NAME}/core-metrics-${NAME}/${NAME}-Temperature-features-bray-curtis.qzv \
  --p-pairwise